In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from absbox.local.util import guess_pool_flow_header

In [3]:
from absbox import API

localAPI = API("http://localhost:8081",lang='english',check=False)

#localAPI = API("https://absbox.org/api/dev",lang='english',check=False)


mortgage = ["Mortgage"
            ,{"originBalance": 12000.0
              ,"originRate": ["fix",0.045]
              ,"originTerm": 36
              ,"freq": "Monthly"
              ,"type": "I_P"
              ,"originDate": "2021-02-01"
              ,"prepayPenalty":{"byTerm":[15,0.1,0.2]}
              }
            ,{"currentBalance": 10000.0
              ,"currentRate": 0.075
              ,"remainTerm": 24
              ,"status": "Current"}]


receivable = ["Invoice"
              ,{"start":"2024-04-01","originBalance":2000
                 ,"originAdvance":1500,"dueDate":"2024-06-01"}
              ,{"status":"Current"}]


receivable1 = ["Invoice"
              ,{"start":"2024-04-01","originBalance":2000
                 ,"originAdvance":1500,"dueDate":"2024-08-01"
                ,"feeType":("Fixed",150)}
              ,{"status":"Current"}]

receivable2 = ["Invoice"
              ,{"start":"2024-04-01","originBalance":2000
                 ,"originAdvance":1500,"dueDate":"2024-06-01"
                ,"feeType":("FixedRate",0.1)}
              ,{"status":"Current"}]

receivable3 = ["Invoice"
              ,{"start":"2024-04-01","originBalance":2000
                 ,"originAdvance":1500,"dueDate":"2024-06-01"
                ,"feeType":("AdvanceRate", 0.12)}
              ,{"status":"Current"}]

receivable4_0 = ["Invoice"
              ,{"start":"2024-04-01","originBalance":2000
                 ,"originAdvance":1500,"dueDate":"2024-06-01"
                ,"feeType":("FactorFee", 0.01, 25, "Up")}
              ,{"status":"Current"}]

receivable4_1 = ["Invoice"
              ,{"start":"2024-04-01","originBalance":2000
                 ,"originAdvance":1500,"dueDate":"2024-06-01"
                ,"feeType":("FactorFee", 0.01, 25, "Down")}
              ,{"status":"Current"}]

receivable5 = ["Invoice"
              ,{"start":"2024-04-01","originBalance":2000
                 ,"originAdvance":1500,"dueDate":"2024-06-01"
                ,"feeType":("CompoundFee"
                            ,("AdvanceRate", 0.12)
                            ,("Fixed",150)
                            )}
              ,{"status":"Current"}]

assets = [receivable4_1]
    
mortgagePoolAssump = ("Pool"
                        ,("Mortgage",None ,{"CPR":0.1}, None, None)
                         ,None
                         ,None)


myAssump = [{"CDR":[0.01,0.0]},{"CPR":[0.01,]}]



receivableAssump = ("Pool"
                   ,("Receivable", {"CDR":0.1}, None, None)
                   ,None
                   ,None)


p = localAPI.runAsset("2021-04-01"
                      ,assets
                      ,poolAssump=receivableAssump
                      ,read=True)
p[0]

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.26.4, server:0.28.1

>>> [{'tag': 'ReceivableFlow', 'contents': ['2024-04-01', 2000, 0, 0, 0, 0, 0, 0, [0, 0, 0, 0, 0, 0]]}, {'tag': 'ReceivableFlow', 'contents': ['2024-06-01', 0, 0, 1925.8, 39.3, 34.9, 0, 34.9, [0, 0, 0, 34.9, 0, 34.9]]}]


,Balance,AccuredFee,Principal,Fee,Default,Recovery,Loss,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,
2024-04-01,2000,0,0.0,0.0,0.0,0,0.0,0,0,0,0.0,0,0.0
2024-06-01,0,0,1925.8,39.3,34.9,0,34.9,0,0,0,34.9,0,34.9


In [53]:
mypool = {'assets':[receivable1,receivable4]
          ,'cutoffDate':"2021-04-01"}

receivableAssump1 = ("ByIndex"
                      ,([0],(("Receivable", {"CDR":0.12}, None, None)
                             ,None,None))
                      ,([1],(("Receivable", "DefaultAtEnd", None, None)
                             ,None,None))
                      )

p = localAPI.runPool(mypool,poolAssump=receivableAssump1)
p[0]

,Balance,AccuredFee,Principal,Fee,Default,Recovery,Loss,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,
2024-04-01,4000,0,0.00,0.00,0.00,0,0.00,0,0,0,0.00,0,0.00
2024-06-01,2000,0,0.00,0.00,2000.00,0,2000.00,0,0,0,2000.00,0,2000.00
2024-08-01,0,0,1772.63,143.72,83.65,0,83.65,0,0,0,2083.65,0,2083.65


In [46]:
from absbox import Generic

test01 = Generic(
    "TEST01"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[receivable1,receivable4]}
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":1000
             ,"rate":0.07
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30}}),)
    ,{"amortizing":[
         ["payFee","acc01",['trusteeFee']]
         ,["accrueAndPayInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payIntResidual","acc01","B"]
     ]}
    ,[["CollectedPrincipal","acc01"]
      ,["CollectedFeePaid","acc01"]]
    ,None
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
    )

In [47]:
r = localAPI.run(test01
             ,runAssump=[]
             ,poolAssump = ("ByIndex"
                              ,([0],(("Receivable", {"CDR":0.12}, None, None)
                                     ,None,None))
                              ,([1],(("Receivable", "DefaultAtEnd", None, None)
                                     ,None,None))
                              )
             ,read=True)

Warning Message from server:
Bond B is not paid off

In [48]:
r['accounts']['acc01'].loc["2024-08-31"]

,balance,change,memo
date,,,
2024-08-31,143.72,143.72,<Pool:CollectedFeePaid>
2024-08-31,1916.35,1772.63,<Pool:CollectedPrincipal>


In [49]:
r['accounts']['acc01'].loc["2024-09-20"]

,balance,change,memo
date,,,
2024-09-20,1886.35,-30.00,<SeqPayFee:trusteeFee>
2024-09-20,1630.32,-256.03,<PayInt:A1>
2024-09-20,630.32,-1000.00,<PayPrin:A1>
2024-09-20,0.00,-630.32,<PayPrin:B>
2024-09-20,0.00,0.00,<PayYield:B>


In [50]:
r['pool']['flow']

,Balance,AccuredFee,Principal,Fee,Default,Recovery,Loss,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,
2021-06-15,4000,0,0.00,0.00,0.00,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-30,4000,0,0.00,0.00,0.00,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-31,4000,0,0.00,0.00,0.00,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-31,4000,0,0.00,0.00,0.00,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN
2021-09-30,4000,0,0.00,0.00,0.00,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-31,4000,0,0.00,0.00,0.00,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-30,4000,0,0.00,0.00,0.00,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-31,4000,0,0.00,0.00,0.00,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-31,4000,0,0.00,0.00,0.00,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN
